<a href="https://colab.research.google.com/github/priyanka25112/pri/blob/main/Miniproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pillow numpy emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.1 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving input.png to input.png


In [4]:
from PIL import Image
import numpy as np
import emoji

# Step 1: Emoji mapping
emoji_map = {'A': '😀', 'B': '😁', 'C': '😂', 'D': '🤣', 'E': '😃', 'F': '😄', 'G': '😅',
    'H': '😆', 'I': '😉', 'J': '😊', 'K': '😋', 'L': '😎', 'M': '😍', 'N': '😘',
    'O': '😗', 'P': '😙', 'Q': '😚', 'R': '🙂', 'S': '🤗', 'T': '🤩', 'U': '🤔',
    'V': '🤨', 'W': '😐', 'X': '😑', 'Y': '😶', 'Z': '🙄',
    'a': '😏', 'b': '😣', 'c': '😥', 'd': '😮', 'e': '🤐', 'f': '😯', 'g': '😪',
    'h': '😫', 'i': '🥱', 'j': '😴', 'k': '😌', 'l': '😛', 'm': '😜', 'n': '😝',
    'o': '🤤', 'p': '😒', 'q': '😓', 'r': '😔', 's': '😕', 't': '🙃', 'u': '🤑',
    'v': '😲', 'w': '☹️', 'x': '🙁', 'y': '😖', 'z': '😞',
    '0': '😟', '1': '😤', '2': '😢', '3': '😭', '4': '😦', '5': '😧', '6': '😨',
    '7': '😩', '8': '🤯', '9': '😬', ' ': '⬜' }
reverse_map = {v: k for k, v in emoji_map.items()}

def text_to_emoji(text):
    return ''.join(emoji_map.get(ch, ch) for ch in text)

def emoji_to_text(emoji_str):
    return ''.join(reverse_map.get(ch, ch) for ch in emoji_str)

# Convert text (UTF-8) to binary
def message_to_binary(msg):
    binary_data = ''.join(format(byte, '08b') for byte in msg.encode('utf-8'))
    return binary_data

# Convert binary to text (UTF-8)
def binary_to_message(binary_str):
    bytes_list = [int(binary_str[i:i+8], 2) for i in range(0, len(binary_str), 8)]
    return bytes(bytes_list).decode('utf-8', errors='ignore')

# LSB Encode
def encode_image(img_path, message, output_path):
    img = Image.open(img_path)
    np_img = np.array(img).astype(np.int32)  # Avoid overflow
    binary_message = message_to_binary(message) + '1111111111111110'  # EOF marker
    data_index = 0
    for row in np_img:
        for pixel in row:
            for channel in range(3):
                if data_index < len(binary_message):
                    pixel[channel] = (pixel[channel] & ~1) | int(binary_message[data_index])
                    data_index += 1
    encoded_img = Image.fromarray(np_img.astype(np.uint8))
    encoded_img.save(output_path)

# LSB Decode
def decode_image(img_path):
    img = Image.open(img_path)
    np_img = np.array(img).astype(np.int32)
    binary_data = ''
    for row in np_img:
        for pixel in row:
            for channel in range(3):
                binary_data += str(pixel[channel] & 1)
                if binary_data[-16:] == '1111111111111110':
                    binary_data = binary_data[:-16]
                    return binary_to_message(binary_data)
    return binary_to_message(binary_data)

# Example Run
secret_text = "Data Science" # Original text
emoji_secret = text_to_emoji(secret_text)  # Convert to emojis
encode_image("input.png", emoji_secret, "stego.png")
decoded_emoji_string = decode_image("stego.png")  # Get back emojis as string
decoded_text = emoji_to_text(decoded_emoji_string)  # Convert emojis to original text


print("Original Text:", secret_text)
print("Emoji Encoded:", emoji_secret)
print("Decoded Text:", decoded_text)
print("Decoded Emojis:", decoded_emoji_string)


Original Text: Data Science
Emoji Encoded: 🤣😏🙃😏⬜🤗😥🥱🤐😝😥🤐
Decoded Text: Data Science
Decoded Emojis: 🤣😏🙃😏⬜🤗😥🥱🤐😝😥🤐
